In [ ]:
!pip install qutip
from qutip import *
import numpy as np
import matplotlib.pyplot as plt

zsh:1: command not found: pip


# Generation of spin-photon entangled pair via BK protocol


Based on the spin-photon entanglement result, we would like to develope the protocol to entangle the two spin systems with high fidelity. To start with, let's prepare two copies of spin-photon systems. In BK protocol (Phys. Rev. A 71, 060310 (2005)), there are 5 steps to achieve high fiedlity or highly entangled state and consists of 5 steps.

**Step 1:** Prepare the initial state as the equally suerposition state as: $|\psi_0\rangle=\frac{1}{\sqrt{2}}\left(|g_{\downarrow}0\rangle_A+|g_{\uparrow}0\rangle_A\right)\otimes\frac{1}{\sqrt{2}}\left(|g_{\downarrow}0\rangle_B+|g_{\uparrow}0\rangle_B\right)$.

**Step 2:** Send the optical field, whose Rabi frequency is $\Omega_s$ and is resonant to the frequency between the transition $|g_\downarrow\rangle_{s}\rightarrow|e\rangle_{s}$, to the NV system, where $s=A,B$. The interaction Hamiltonian can be written as: $$\hat{H}/\hbar=\sum_{s=A,B}\Omega_s\hat{a}_s\hat{\sigma}_{+,s}+\text{h.c.}$$
, where $\hat{\sigma}_{+,s}=|e\rangle_{ss}\langle g_{\downarrow}|$, with the excitation time-dependent Hamiltonian that uses the semi-classical approximation: $$\hat{H}_{\text{exc}}\left(t\right)/\hbar=\sum_{s=A,B}P_s\left(t\right)\hat{\sigma}_{+,s}+\text{h.c.}$$
, where $P_s\left(t\right)$ is the Rabi frequency of the excitation pulse. For simplicity, let's assume identical Rabi frequencies, excitation pulses of system A and B as $\Omega$ and $P\left(t\right)$.

**Step 3:** Counts the clicks that occur at detector $D_1$ or $D_2$. As a reminder, $D_1$ (or $D_2$) stands for the collapsing operator of mode $\hat{a}_A+\hat{a}_B$ (or $\hat{a}_A-\hat{a}_B$).

**Step 4:** Applying both qubits (A and B) with X-gate operation individually, $\hat{X}=\left(|g_\uparrow\rangle\langle g_\downarrow|+|g_\downarrow\rangle\langle g_\uparrow|+|e\rangle\langle e|\right)_A\otimes\left(|g_\uparrow\rangle\langle g_\downarrow|+|g_\downarrow\rangle\langle g_\uparrow|+|e\rangle\langle e|\right)_B$, to flip the spins.

**Step 5:** Repeat Step 1.

In the following, we provide the codes to simulate the overall process.

# Physical Parameters

In [ ]:
#Deminsions
N = 5 # Set where to truncate Fock state of cavity
Ne = 3 # number of electronic levels
Nn = 2 # number of nuclear levels


#Inefficiency
kappa =0.1 # Cavity decay rate
gamma =1 # Atomic decay rate
K_c = 20 # rate for collapsing to detector
g0 = 2 # coupling strength (Rabi frequency of vacuum field) between photon and e-spin
theta=np.pi/4 # Initial e-spin state angle
gamma_dec=0.005 # dephasing rate of e-spin
g_hyper = np.pi/2 # hyperfine interaction strength


#Excitation pulse
#Opitcal Pi-pulse
center_optical=0.5
life_time_optical=0.04
peak_optical =np.sqrt(np.pi)/2/life_time_optical

#Microwave Pi-pulse
center_microwave=2
sig_microwave=0.4

#Dephasing standard deviation
sig_dephase=np.pi/4

# Simulationm parameters

In [ ]:
time_scale=1000
numb=50 # numbers of trajectories

numb_nuclear=10 # numbers of trajectories of hyperfine interaction

t_optical = np.linspace(0.0, 5.0, time_scale) # Evolutionary time of optical transition
t_microwave = t_optical # Evolutionary time of microwave transition
t_hyperfine = np.linspace(0.0, 1, time_scale) # Evolutionary time of hyperfine interaction


# Other parameters

In [ ]:
psi = [[] for _ in range(4)] #Storing the state of the resulting state under BK-scheme
cluster_2Qubit = [[] for _ in range(4)] #Storing the resulting cluster state
counts = [0 for _ in range(4)] # Counting the numbers of clicks
rho_final = [0 for _ in range(4)] #Storing the state for calculating the entanglement entropy

# Functions

In [ ]:
def optical_pulse(t,arg):
    return peak_optical * np.exp(-((t-center_optical) / life_time_optical) ** 2)

def decoh(phi):
    ele = ground_e_down*ground_e_down.dag()+np.exp(1j*phi)*ground_e_up*ground_e_up.dag()+excited_e*excited_e.dag()
    return tensor(qeye(N),ele,qeye(Nn))

def microwave_pulse(t,phi,arg):
    return phi*(np.exp(-(t-center_microwave)**2/sig_microwave**2/2)/sig_microwave/np.sqrt(2*np.pi))

# Operators

In [ ]:
ground_e_down = basis(Ne, 0)
ground_e_up = basis(Ne, 1)
excited_e = basis(Ne, 2)

ground_n_down = basis(Nn, 0)
ground_n_up = basis(Nn, 1)

def create_operators(n_systems):
    assert 2 <= n_systems <= 26, "Invalid number of systems"

    systems = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"[:n_systems]
    module_identity = [qeye(N), qeye(Ne), qeye(Nn)]

    ops = dict()
    for i, l in enumerate(systems):

        sigma_e_L_gd_e = module_identity * n_systems
        sigma_e_L_gd_e[3 * i + 1] = ground_e_down * excited_e.dag()
        ops[f"sigma_e_{l}_gd_e"] = tensor(*sigma_e_L_gd_e)

        sigma_e_L_gd_gu = module_identity * n_systems
        sigma_e_L_gd_gu[3 * i + 1] = ground_e_down * ground_e_up.dag()
        ops[f"sigma_e_{l}_gd_gu"] = tensor(*sigma_e_L_gd_gu)

        sigma_n_L_gd_gu = module_identity * n_systems
        sigma_n_L_gd_gu[3 * i + 2] = ground_n_down * ground_n_up.dag()
        ops[f"sigma_n_{l}_gd_gu"] = tensor(*sigma_n_L_gd_gu)

        a_L = module_identity * n_systems
        a_L[3 * i] = destroy(N)
        ops[f"a_{l}"] = tensor(*a_L)

        sigma_e_L_gd_gu, sigma_n_L_gd_gu = ops[f"sigma_e_{l}_gd_gu"], ops[f"sigma_n_{l}_gd_gu"]
        H0_hyper_L = -g_hyper * (sigma_e_L_gd_gu.dag() * sigma_n_L_gd_gu + sigma_n_L_gd_gu.dag() * sigma_e_L_gd_gu)
        ops[f"H0_hyper_{l}"] = H0_hyper_L

    #Letters for the last two systems, which will be involved in the entanglement generation
    l1, l2 = systems[-2], systems[-1]
    sigma_e_1_gd_e, sigma_e_2_gd_e = ops[f"sigma_e_{l1}_gd_e"], ops[f"sigma_e_{l2}_gd_e"]
    sigma_e_1_gd_gu, sigma_e_2_gd_gu = ops[f"sigma_e_{l1}_gd_gu"], ops[f"sigma_e_{l2}_gd_gu"]
    a_1, a_2 = ops[f"a_{l1}"], ops[f"a_{l2}"]

    # Optical photon and e-spin
    H0_o_1 = -g0 * (sigma_e_1_gd_e.dag() * a_1+a_1.dag()* sigma_e_1_gd_e) # time-independent Hamiltonian of system A
    H0_o_2 = -g0 * (sigma_e_2_gd_e.dag() * a_2+a_2.dag()* sigma_e_2_gd_e) # time-independent Hamiltonian of system B
    H0_o = H0_o_1 + H0_o_2 # time-independent Hermitian of global system
    H1_o_1 = (sigma_e_1_gd_e.dag() + sigma_e_1_gd_e) # time-dependent Hamiltonian of system A after semi-classical approximation
    H1_o_2 = (sigma_e_2_gd_e.dag() + sigma_e_2_gd_e) # time-dependent Hamiltonian of system A after semi-classical approximation
    H_optical = [H0_o,[H1_o_1, optical_pulse],[H1_o_2,optical_pulse]]
    ops["H_optical"] = H_optical

    # Microwave photon and e-spin (X-gate)
    H1_m_1 = (sigma_e_1_gd_gu.dag() + sigma_e_1_gd_gu) # time-dependent Hamiltonian of system A after semi-classical approximation
    H1_m_2 = (sigma_e_2_gd_gu.dag() + sigma_e_2_gd_gu) # time-dependent Hamiltonian of system A after semi-classical approximation
    H_microwave=[[H1_m_1,microwave_pulse(t_microwave,np.pi/2,None)], [H1_m_2,microwave_pulse(t_microwave,np.pi/2,None)]]
    ops["H_microwave"] = H_microwave

    # Hyperfine interaction between e-spin and n-spin
    H_hyper = None
    for l in systems:
        H0_hyper_L = ops[f"H0_hyper_{l}"]
        if H_hyper == None: H_hyper = H0_hyper_L
        else: H_hyper += H0_hyper_L
    ops["H_hyper"] = H_hyper

    # This part is for the generation of cluster state with X and Hadmard gates
    sigma_n_2_gd_gu = ops[f"sigma_n_{l2}_gd_gu"]
    H1_m_cluster_B = -1j*(sigma_n_2_gd_gu.dag() - sigma_n_2_gd_gu) # time-dependent Hamiltonian of system A after semi-classical approximation
    H_had=[H1_m_cluster_B,microwave_pulse(t_microwave,np.pi/4,None)]
    ops["H_had"] = H_had

    c_ops = []
    c_ops.append(np.sqrt(kappa) * a_1) # Cavity decay of system A. C0
    c_ops.append(np.sqrt(kappa) * a_2) # Cavity decay of system B. C1
    for l in systems:
        c_ops.append(np.sqrt(gamma) * ops[f"sigma_e_{l}_gd_e"])
    c_ops.append(np.sqrt(K_c) * (a_1+a_2)/np.sqrt(2)) # Collapsing to detector 1        C4
    c_ops.append(np.sqrt(K_c) * (a_1-a_2)/np.sqrt(2)) # Collapsing to detector 2        C5
    ops["c_ops"] = c_ops

    c_ops_dephasing = []
    c_ops_had=[]
    for _ in range(20):
        for i in range(n_systems):
            terms = module_identity * n_systems
            c_ops_dephasing.append(np.sqrt(gamma_dec) * tensor(*terms[:i*3], decoh(np.random.normal(0,sig_dephase)), *terms[(i+1)*3:]))
        terms = module_identity * (n_systems - 1)
        terms.append(decoh(np.random.normal(0,sig_dephase)))
        c_ops_had.append(np.sqrt(gamma_dec)*tensor(*terms)) ### This part is for cluster state generation
    ops["c_ops_dephasing"] = c_ops_dephasing
    ops["c_ops_had"] = c_ops_had

    return ops


# Simulations

In [ ]:
operators = create_operators(2)
H_optical, H_microwave, H_hyper, H_had = operators["H_optical"], operators["H_microwave"], operators["H_hyper"], operators["H_had"]
c_ops, c_ops_dephasing, c_ops_had = operators["c_ops"], operators["c_ops_dephasing"], operators["c_ops_had"]

photon = basis(N, 0) # Initial photonic state
spin_e = ground_e_down*np.cos(theta)+ground_e_up*np.sin(theta) # Initial electronic spin state
spin_n = ground_n_down
psi_initial = tensor(photon, spin_e, spin_n, photon, spin_e, spin_n) # Initial global state

output = mcsolve(H_optical, psi_initial, t_optical, c_ops, [], ntraj = numb, progress_bar=False)

We count the clicks that occur on each detector, and run the monte carlo solver a second time, recording the average resulting states:

In [ ]:
def run_second_mcsolve(final_state, first_run_detector, counts):
    #Step 4
    # Let's apply the pi pulse to flip the two spins, which is equivalent to apply a deterministic X gate
    result0 = mcsolve(H_microwave, final_state, t_microwave, c_ops_dephasing, [], ntraj = 1, progress_bar=False)
    result = mcsolve(H_optical, result0.states[0][-1], t_optical, c_ops, [], ntraj = numb, progress_bar=False)

    for i in range(numb):
        #For all trajectories, check if either detector fired, and if so add the final state to the appropriate index in rhos
        if 4 in result.col_which[i]:
            psi_num = 0 if first_run_detector == 4 else 2
            psi[psi_num].append(result.states[i][-1])
            counts[psi_num] += 1
        if 5 in result.col_which[i]:
            psi_num = 1 if first_run_detector == 4 else 3
            psi[psi_num].append(result.states[i][-1])
            counts[psi_num] += 1

for i in range(numb):
    #Step 3
    if 4 in output.col_which[i]:
        run_second_mcsolve(output.states[i][-1], 4, counts)
    if 5 in output.col_which[i]:
        run_second_mcsolve(output.states[i][-1], 5, counts)

for i in range(4):
    rhos = 0
    for j in range(len(psi[i])):
        final_result = mcsolve(H_hyper, psi[i][j], t_hyperfine, c_ops_dephasing, ntraj = numb_nuclear, progress_bar=False)
        final_result1 = mcsolve(H_had, final_result.states[0][-1], t_microwave, c_ops_had, [], ntraj = 1, progress_bar=False)
        cluster_2Qubit[i].append(final_result1.states[0][-1])
        for k in range(numb_nuclear):
            rhos += final_result.states[k][-1].ptrace([2,5])
    rho_final[i]=rhos/numb_nuclear/len(psi[i])

In [ ]:
# We expect the spin state that D1 fires in both rounds will result in a bell state |01>+|10> and let's calculate the entanglement entropy as well as their fiedlities
Bell1=tensor(ground_n_down,ground_n_up)/np.sqrt(2)+tensor(ground_n_up,ground_n_down)/np.sqrt(2)
print("BK scheme:  entanglement entropy (D1->D1): ",(entropy_vn(rho_final[0].ptrace(1),2)+entropy_vn(rho_final[0].ptrace(0),2))/2, " with fidelity:",fidelity(rho_final[0],Bell1))
# We expect the spin state that D1 fires in both rounds will result in a bell state |01>+|10> and let's calculate the entanglement entropy as well as their fiedlities
print("BK scheme:  entanglement entropy (D2->D2): ",(entropy_vn(rho_final[3].ptrace(1),2)+entropy_vn(rho_final[3].ptrace(0),2))/2, " with fidelity:",fidelity(rho_final[3],Bell1))
# We expect the spin state of the event that D1 fires in the first and D2 fires in the second round results in a bell state |01>-|10> and let's calculate the entanglement entropy as well as their fiedlities
Bell2=tensor(ground_n_down,ground_n_up)/np.sqrt(2)-tensor(ground_n_up,ground_n_down)/np.sqrt(2)
print("BK scheme:  entanglement entropy (D1->D2): ",(entropy_vn(rho_final[1].ptrace(1),2)+entropy_vn(rho_final[1].ptrace(0),2))/2, " with fidelity:",fidelity(rho_final[1],Bell2))
# We expect the spin state of the event that D2 fires in the first and D1 fires in the second round results in a bell state |01>-|10> and let's calculate the entanglement entropy as well as their fiedlities
print("BK scheme:  entanglement entropy (D2->D1): ",(entropy_vn(rho_final[2].ptrace(1),2)+entropy_vn(rho_final[2].ptrace(0),2))/2, " with fidelity:",fidelity(rho_final[2],Bell2))

BK scheme:  entanglement entropy (D1->D1):  0.9998710000175292  with fidelity: 0.8851970353470543
BK scheme:  entanglement entropy (D2->D2):  0.9988272964791314  with fidelity: 0.9475004097615484
BK scheme:  entanglement entropy (D1->D2):  0.9998863388271505  with fidelity: 0.8912809395809351
BK scheme:  entanglement entropy (D2->D1):  0.9981272757007833  with fidelity: 0.9314339828485343


# 2-qubit cluster (D1->D1)

In [ ]:
Z = ground_n_down*ground_n_down.dag() - ground_n_up*ground_n_up.dag()
plus = ground_n_down/np.sqrt(2)+ground_n_up/np.sqrt(2)
minus = Z * plus
Cluster_2qubit_ideal = tensor(plus,ground_n_down)/np.sqrt(2) + tensor(minus,ground_n_up)/np.sqrt(2)

entanglement = np.zeros(len(cluster_2Qubit[0]))

for i in range(len(cluster_2Qubit[0])):
    rho_2cluster = cluster_2Qubit[0][i].ptrace([2,5])
    entanglement[i] = fidelity(rho_2cluster, Cluster_2qubit_ideal)

print("Max fidelity: ",np.max(entanglement))
print("Average fidelity: ",np.mean(entanglement))
print("Min fidelity: ",np.min(entanglement))

Max fidelity:  1.0000000107832956
Average fidelity:  0.8979475661674206
Min fidelity:  0.10509313474784916


# 3-qubit cluster

In [ ]:
module_identity = tensor(qeye(N), qeye(Ne), qeye(Nn))
module_basis = tensor(basis(N,0), basis(Ne,0), basis(Nn,0))

# After successfully generating the electronic spin entanglement, let's store the entanglement of electronic spin with nuclear spin
psi_0 = tensor(cluster_2Qubit[0][0],module_basis) # Initial global state

operators = create_operators(3)